# Statistical Mechanics Python Template

Here are some helper functions for doing statistical mechanics calculations and plotting. Remember that once we know the energies and degeneracies (locations of all the microstates on our energy diagram), we do the steps

1. Calculate the ways for each state or level using the Boltzmann distribution.
2. Calculate the total ways $q$ (molecular partition function).
3. Calculate anything else we need from 1, 2, and the given information about the system.

### Imports

Let's import all the packages we might need:

In [1]:
# Usual functions for math: np.exp, np.linspace, etc...
import numpy as np

# Spreadsheet-style tables (with data in rows and labeled columns) in Python
import pandas as pd

# dictionaries that can use a.b rather than a['b'] (easier to type)
from munch import Munch

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px

# An easy way to make things interactive...
from ipywidgets import interact

# Physical constants
kB = 1.381e-23 # J/K
h = 6.626e-34 # Js
R = 8.314 # J/mol-K
R_kJ = R / 1000 # kJ/mol-K

# Units; add any other conversion factors you need 
amu = 1.6605e-27 # 1 amu = 1.6605e-27 kg

## Statistical mechanics calculations
### Case 1: Just a few energy levels

There are lots of ways to do these calculations - here's one approach. Just define a function that takes the problem parameters as inputs, and then do all your calculations and output them all in a dictionary (or Munch for slightly easier access with `.`).

#### Example 
Consider the 4-unit polymer model from class. We had two energy levels that correspond to two macrostates:
- The bonded (B) macrostate with 2 microstates (degeneracy $g_B = 2$) at an energy of $0$
- The unbonded (U) macrostate with 5 microstates (degeneracy $g_U = 5$) at an energy of $\varepsilon$

**What are the inputs?** (trick: consider $k_B$ or $R$ an input so that you can change the units of your energy from J/particle to kJ/mol easily - $\varepsilon/k_B T$ = $E/RT$)

Enter inputs here...

**Example function:** Now that we have our inputs, let's go through the steps in a function.

In [2]:
#

In [3]:
def organic(T, R=R_kJ):
    """The 4 unit polymer model from class; by default, the model uses an energy E in kJ/mol.
    If you pass R=kB, you can use J/particle instead."""
    # First define the ways
    eB = 96.47696477*0.04
    eC = 96.47696477*0.05
    wA = 1
    wB = 3*np.exp(-eB/(R*T))
    wC = 3*np.exp(-eC/(R*T))
    
    # Define the total ways q...
    q = wA + wB + wC
    
    # Define anything else we need
    Kcb = wC/wB
    Kba = wB / wA
    pB = wB/q
    pA = wA/q
    pC = wC/q
    avg_energy = 0.04*pB + 0.05*pC

    # Outputs: Easy, we'll output everything!
    return Munch(locals()) # locals is a dictionary containing all variables defined in the function

out = organic(5, 273) # 5 kJ/mol, T = 273 K
out

Munch({'T': 5, 'R': 273, 'eB': 3.8590785908, 'eC': 4.8238482385, 'wA': 1, 'wB': 2.9915304865838257, 'wC': 2.9894168469458515, 'q': 6.980947333529677, 'Kcb': 0.9992934587671918, 'Kba': 2.9915304865838257, 'pB': 0.4285278693072822, 'pA': 0.1432470339945086, 'pC': 0.4282250966982092, 'avg_energy': 0.03855236960720175})

In [9]:
def dna(E, T, R=R_kJ):
    """The 4 unit polymer model from class; by default, the model uses an energy E in kJ/mol.
    If you pass R=kB, you can use J/particle instead."""
    # First define the ways
    T = np.array(T)
    ways = np.c_[[3**i * np.exp(-i*E/(R*T)) for i in range(5)]]
    
    # Define the total ways q...
    q = np.sum(ways, axis=0)
    
#     # Define anything else we need
    p = ways/q.reshape((-1, 1))

    # Outputs: Easy, we'll output everything!
    return Munch(locals()) # locals is a dictionary containing all variables defined in the function

out = dna(5, 310) # 5 kJ/mol, T = 273 K
out

Munch({'ways': array([[1.        ],
       [0.43111847],
       [0.18586314],
       [0.08012903],
       [0.03454511]]), 'q': array([1.73165575]), 'p': array([[0.57748199],
       [0.24896315],
       [0.10733261],
       [0.04627307],
       [0.01994918]]), 'E': 5, 'R': 0.008314, 'T': array(310)})

### Plotting

Once we have our function, we can call our function using a numpy array of temperature to quickly determine how all of our ways, probabilities, and equilibrium constants vary with temperature.

In [23]:
temps = np.linspace(1, 800, num=200) # num is the number of data points
outs = organic(temps) # Not a great name - just all of our outputs

In [24]:
# Plotly plot
px.line(x=outs.T, y=[outs.Kba, outs.Kcb], labels={'x': "Temperature (K)", 'y': 'Ways'})

### Pandas dataframe

This will look like an Excel table; plotly knows how to plot dataframes by default.

In [25]:
df = pd.DataFrame(outs)
df

,T,R,eB,eC,wA,wB,wC,q,Kcb,Kba,pB,pA,pC,avg_energy
0,1.000000,0.008314,3.859079,4.823848,1,7.802970e-202,3.133607e-252,1.000000,4.015916e-51,7.802970e-202,7.802970e-202,1.000000,3.133607e-252,3.121188e-203
1,5.015075,0.008314,3.859079,4.823848,1,1.911359e-40,1.707645e-50,1.000000,8.934192e-11,1.911359e-40,1.911359e-40,1.000000,1.707645e-50,7.645436e-42
2,9.030151,0.008314,3.859079,4.823848,1,1.424259e-22,3.738574e-28,1.000000,2.624925e-06,1.424259e-22,1.424259e-22,1.000000,3.738574e-28,5.697055e-24
3,13.045226,0.008314,3.859079,4.823848,1,1.057676e-15,1.449307e-19,1.000000,1.370275e-04,1.057676e-15,1.057676e-15,1.000000,1.449307e-19,4.231428e-17
4,17.060302,0.008314,3.859079,4.823848,1,4.582489e-12,5.094440e-15,1.000000,1.111719e-03,4.582489e-12,4.582489e-12,1.000000,5.094440e-15,1.835543e-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,783.939698,0.008314,3.859079,4.823848,1,1.659503e+00,1.431173e+00,4.090675,8.624108e-01,1.659503e+00,4.056794e-01,0.244458,3.498622e-01,3.372029e-02
196,787.954774,0.008314,3.859079,4.823848,1,1.664517e+00,1.436580e+00,4.101097,8.630615e-01,1.664517e+00,4.058711e-01,0.243837,3.502917e-01,3.374943e-02
197,791.969849,0.008314,3.859079,4.823848,1,1.669495e+00,1.441953e+00,4.111449,8.637061e-01,1.669495e+00,4.060601e-01,0.243223,3.507166e-01,3.377824e-02
198,795.984925,0.008314,3.859079,4.823848,1,1.674438e+00,1.447292e+00,4.121730,8.643447e-01,1.674438e+00,4.062465e-01,0.242617,3.511370e-01,3.380671e-02


In [26]:
# If you give plotly a dataframe, just use the names of the columns to tell plotly what should be on each axis
# If you give a list, it will plot multiple things
px.line(df, x='T', y=['pA', 'pB', 'pC'],
        labels={'value': "Ways", # value is the default name given to the y-axis when multiple things are plotted
                'T': "Temperature (K)"}
       )

In [28]:
px.line(df, x='T', y='avg_energy')

In [10]:
sum([7, 20, 21, 7,5])

60